<a href="https://colab.research.google.com/github/aleynakirmizi/st-dbscan/blob/main/purity_cross_entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install ciso8601

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for ciso8601: filename=ciso8601-2.2.0-cp37-cp37m-linux_x86_64.whl size=37537 sha256=ab73e8f357ee4775d9af8ead8506a30960f3bc49ecf06239eaa6df14240b550d
  Stored in directory: /root/.cache/pip/wheels/ad/25/8f/3b0a82303191efe3c1204f3741c42d8eb2b0236567e22485de
Successfully built ciso8601


In [2]:
import ciso8601
import pandas as pd
import time
import re
import string
from sklearn.feature_extraction.text import  CountVectorizer
import nltk
import numpy as np
import json
from st_dbscan import st_dbSCAN

In [3]:
path = '/content/drive/MyDrive/News_Category_Dataset_v2.json'
list_=[]
with open(path) as files:
  for file in files:
    list_.append(json.loads(file))
df = pd.DataFrame(list_).copy()
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [4]:
train_data=df[['category','headline','date']]
train_data['date'] = train_data['date'].apply(lambda x: ciso8601.parse_datetime(x))
train_data['date'] = train_data['date'].apply(lambda x: time.mktime(x.timetuple()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
set_default = train_data.loc[0][2]
set_default

1527292800.0

In [6]:
new_date_list = []
for date in train_data["date"]:
  diff = set_default -date
  diff = round(diff/86400)
  new_date_list.append(diff)

In [7]:
train_data["date"]=new_date_list
train_data = train_data.head(1000)
train_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,category,headline,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,0
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,0
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,0
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,0
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,0
...,...,...,...
995,POLITICS,NY Attorney General Eric Schneiderman Resigns ...,18
996,POLITICS,‘Sanctuary Cities’ Are Working In The Trump Er...,18
997,MEDIA,Shepard Smith: You Can't Say Trump Pulled Out ...,18
998,WEIRD NEWS,Stormy Daniels Finds A Crunchy New Way to Chip...,18


In [8]:
nltk.download('punkt')
nltk.download('stopwords')
def clean_text(text):
    text = text.lower()                                  # lower-case all characters
    text =  re.sub(r'@\S+', '',text)                     # remove twitter handles
    text =  re.sub(r'http\S+', '',text)                  # remove urls
    text =  re.sub(r'pic.\S+', '',text)
    text =  re.sub(r"[^a-zA-Z+']", ' ',text)             # only keeps characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text+' ')      # keep words with length>1 only
    text = "".join([i for i in text if i not in string.punctuation])
    words = nltk.tokenize.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words('english')   # remove stopwords
    text = " ".join([i for i in words if i not in stopwords and len(i)>2])
    text= re.sub("\s[\s]+", " ",text).strip()            # remove repeated/leading/trailing spaces
    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
train_data['Text_cleaning'] = train_data.headline.apply(clean_text)
train_data['tokenized_sents'] = train_data.apply(lambda row: nltk.word_tokenize(row['Text_cleaning']), axis=1)
data=train_data.loc[:,['date']].values
df= pd.DataFrame()
df["vector_text"] = train_data['tokenized_sents']
df["date"] = data
df

,vector_text,date
0,"[mass, shootings, texas, last, week]",0
1,"[smith, joins, diplo, nicky, jam, world, cups,...",0
2,"[hugh, grant, marries, first, time, age]",0
3,"[jim, carrey, blasts, castrato, adam, schiff, ...",0
4,"[julianna, margulies, uses, donald, trump, poo...",0
...,...,...
995,"[attorney, general, eric, schneiderman, resign...",18
996,"[sanctuary, cities, working, trump, era, surpr...",18
997,"[shepard, smith, cant, say, trump, pulled, ira...",18
998,"[stormy, daniels, finds, crunchy, new, way, ch...",18


In [10]:
tf = st_dbSCAN(df,0.8,9,8,6)
fit = tf.fit_transform(df)

/content/st_dbscan.py:50: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mean = cluster.mean()


In [11]:
df

,vector_text,date,cluster
0,"[mass, shootings, texas, last, week]",0,-1
1,"[smith, joins, diplo, nicky, jam, world, cups,...",0,-1
2,"[hugh, grant, marries, first, time, age]",0,-1
3,"[jim, carrey, blasts, castrato, adam, schiff, ...",0,-1
4,"[julianna, margulies, uses, donald, trump, poo...",0,-1
...,...,...,...
995,"[attorney, general, eric, schneiderman, resign...",18,-1
996,"[sanctuary, cities, working, trump, era, surpr...",18,-1
997,"[shepard, smith, cant, say, trump, pulled, ira...",18,3
998,"[stormy, daniels, finds, crunchy, new, way, ch...",18,-1


In [12]:
vectorizer = CountVectorizer()
data_vectorizer = vectorizer.fit_transform(train_data['Text_cleaning'])
arr=data_vectorizer.todense()
train_data['vectorized_text']=arr.tolist()
df["vectors"] = train_data["vectorized_text"]
df['category'] = train_data["category"]


In [35]:
df_cluster_1 = df[df["cluster"]==1]
df_cluster_2 = df[df["cluster"]==2]
df_cluster_3 = df[df["cluster"]==3]


In [14]:
df.cluster.value_counts()

-1    963
 2     16
 1     11
 3     10
Name: cluster, dtype: int64

In [16]:
from sklearn.manifold import TSNE

In [17]:
i=1
X_embedded = TSNE(n_components=2,random_state=0).fit_transform(df[df["cluster"]==i]["vectors"].values.tolist())
X_embedded

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


array([[ 158.30424 ,  152.85916 ],
       [-154.90863 ,  -15.818744],
       [-213.98087 ,  376.24023 ],
       [ 378.75882 , -486.59897 ],
       [-381.8619  , -316.0079  ],
       [ 148.38574 ,  530.91614 ],
       [ -51.129013, -527.20276 ],
       [ 137.89061 , -201.22983 ],
       [-511.84128 ,   96.37503 ],
       [ 509.04468 ,  293.87186 ],
       [ 518.29144 ,  -99.62894 ]], dtype=float32)

In [18]:
internal = []
for i in df.cluster.value_counts().index:
  result_list=[]
  sum=0
  if i ==  -1 :
    continue
  else:
    print(i)
    s=df[df["cluster"]==i]
    X_embedded = TSNE(n_components=2,random_state=0).fit_transform(df[df["cluster"]==i]["vectors"].values.tolist())
    cluster_dimensions = [X_embedded[:,0].mean(),X_embedded[:,1].mean()]
    for (i,j) in zip(X_embedded[:,0],X_embedded[:,1]):
      result_x = (cluster_dimensions[0]-i)**2
      result_y = (cluster_dimensions[1]-j)**2
      result = result_x + result_y
      result =result**0.5
      result_list.append(result)
    for k in result_list:
      sum = sum +k
    s = (len(s)-1)
    r = sum / s
    internal.append(r)
print(internal)

2


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


1


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


3


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


[361.2044128643307, 486.24958226861753, 50.941659546178315]


In [19]:
number=len(df_cluster_1["category"].value_counts())
number

3

In [20]:
df_cluster_1["category"].value_counts()
df_cluster_1

,vector_text,date,cluster,vectors,category
10,"[justin, timberlake, visits, texas, school, sh...",0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ENTERTAINMENT
42,"[people, injured, indiana, school, shooting]",1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",CRIME
63,"[things, nra, blamed, texas, school, shooting]",1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS
282,"[kelly, clarkson, calls, moments, action, hono...",5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ENTERTAINMENT
334,"[texas, gov, campaign, cancels, shotgun, givea...",6,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS
341,"[parents, school, shooting, victims, decry, mo...",6,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS
360,"[victims, santa, high, school, shooting]",7,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",CRIME
364,"[jimmy, kimmel, shreds, cowardly, trump, lawma...",7,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS
374,"[santa, high, schoolers, held, gun, violence, ...",8,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS
399,"[twitter, users, pan, ted, cruz, tweets, praye...",8,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS


Categorisi fazla olana bakılacak.

In [21]:
#In category one, politics category is the majority in that cluster. So the label for the cluster is politics.
# The more politics, the more purity for this cluster.
result = []
for i in range(number):
  result.append( df_cluster_1["category"].value_counts()[i] / df_cluster_1["category"].value_counts().sum())
print(max(result))
#the purity/ quality for this cluster is : 

0.6363636363636364


In [22]:
df_cluster_2["category"].value_counts()

WORLD NEWS       8
POLITICS         4
COMEDY           3
ENTERTAINMENT    1
Name: category, dtype: int64

In [23]:
#the majority of the second class is world news.
# it defines the label of that cluster
result = []
for i in range(number):
  result.append( df_cluster_2["category"].value_counts()[i] / df_cluster_2["category"].value_counts().sum())
print(max(result))
print(result)

0.5
[0.5, 0.25, 0.1875]


In [24]:
df_cluster_3["category"].value_counts()

POLITICS      7
WORLD NEWS    2
MEDIA         1
Name: category, dtype: int64

In [25]:
#politics is the majority of the 3rd cluster.
result = []
for i in range(number):
  result.append( df_cluster_3["category"].value_counts()[i] / df_cluster_3["category"].value_counts().sum())
print(max(result))
print(result)

0.7
[0.7, 0.2, 0.1]


categorical crossentropy / preprocessing

In [26]:
df_cluster_1

,vector_text,date,cluster,vectors,category
10,"[justin, timberlake, visits, texas, school, sh...",0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ENTERTAINMENT
42,"[people, injured, indiana, school, shooting]",1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",CRIME
63,"[things, nra, blamed, texas, school, shooting]",1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS
282,"[kelly, clarkson, calls, moments, action, hono...",5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ENTERTAINMENT
334,"[texas, gov, campaign, cancels, shotgun, givea...",6,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS
341,"[parents, school, shooting, victims, decry, mo...",6,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS
360,"[victims, santa, high, school, shooting]",7,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",CRIME
364,"[jimmy, kimmel, shreds, cowardly, trump, lawma...",7,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS
374,"[santa, high, schoolers, held, gun, violence, ...",8,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS
399,"[twitter, users, pan, ted, cruz, tweets, praye...",8,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS


In [31]:
from sklearn.preprocessing import LabelEncoder

In [36]:
le=LabelEncoder()
df_cluster_1["encodded_category"] = le.fit_transform(df_cluster_1["category"])
df_cluster_1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,vector_text,date,cluster,vectors,category,encodded_category
10,"[justin, timberlake, visits, texas, school, sh...",0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ENTERTAINMENT,1
42,"[people, injured, indiana, school, shooting]",1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",CRIME,0
63,"[things, nra, blamed, texas, school, shooting]",1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS,2
282,"[kelly, clarkson, calls, moments, action, hono...",5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ENTERTAINMENT,1
334,"[texas, gov, campaign, cancels, shotgun, givea...",6,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS,2
341,"[parents, school, shooting, victims, decry, mo...",6,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS,2
360,"[victims, santa, high, school, shooting]",7,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",CRIME,0
364,"[jimmy, kimmel, shreds, cowardly, trump, lawma...",7,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS,2
374,"[santa, high, schoolers, held, gun, violence, ...",8,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS,2
399,"[twitter, users, pan, ted, cruz, tweets, praye...",8,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",POLITICS,2


In [37]:
df_cluster_1 = pd.get_dummies(df_cluster_1,columns=["category"],prefix = ["category"])
df_cluster_1

,vector_text,date,cluster,vectors,encodded_category,category_CRIME,category_ENTERTAINMENT,category_POLITICS
10,"[justin, timberlake, visits, texas, school, sh...",0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,1,0
42,"[people, injured, indiana, school, shooting]",1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,0,0
63,"[things, nra, blamed, texas, school, shooting]",1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0,0,1
282,"[kelly, clarkson, calls, moments, action, hono...",5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,1,0
334,"[texas, gov, campaign, cancels, shotgun, givea...",6,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0,0,1
341,"[parents, school, shooting, victims, decry, mo...",6,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0,0,1
360,"[victims, santa, high, school, shooting]",7,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,1,0,0
364,"[jimmy, kimmel, shreds, cowardly, trump, lawma...",7,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0,0,1
374,"[santa, high, schoolers, held, gun, violence, ...",8,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0,0,1
399,"[twitter, users, pan, ted, cruz, tweets, praye...",8,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0,0,1


In [59]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [60]:
category = df_cluster_1.iloc[:,5:8].values.tolist()
category

[[0, 1, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1]]

softmax output of cluster 1 and categorical cross entropy

In [82]:
softmax(category).tolist()

[[0.0692685616719318, 0.1882914724763069, 0.043425446465446674],
 [0.1882914724763069, 0.0692685616719318, 0.043425446465446674],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476],
 [0.0692685616719318, 0.1882914724763069, 0.043425446465446674],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476],
 [0.1882914724763069, 0.0692685616719318, 0.043425446465446674],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476]]

In [64]:
label = df_cluster_1["category_POLITICS"].values.tolist()
label

[0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1]

In [ ]:
softmax(label).tolist()

In [109]:
label = df_cluster_1["encodded_category"]
label

10     1
42     0
63     2
282    1
334    2
341    2
360    0
364    2
374    2
399    2
421    2
Name: encodded_category, dtype: int64

In [87]:
category =[ [0.0692685616719318, 0.1882914724763069, 0.043425446465446674],
 [0.1882914724763069, 0.0692685616719318, 0.043425446465446674],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476],
 [0.0692685616719318, 0.1882914724763069, 0.043425446465446674],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476],
 [0.1882914724763069, 0.0692685616719318, 0.043425446465446674],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476],
 [0.0692685616719318, 0.0692685616719318, 0.11804260201974476]]

In [88]:
log_loss(label,category)

0.664640037841504

sofmax for cluster 2 and categorical cross entropy 

In [ ]:
le=LabelEncoder()
df_cluster_2["encodded_category"] = le.fit_transform(df_cluster_2["category"])
df_cluster_2

In [ ]:
df_cluster_2 = pd.get_dummies(df_cluster_2,columns=["category"],prefix = ["category"])
df_cluster_2

In [ ]:
category_2 = df_cluster_2.iloc[:,5:9].values.tolist()
category_2

In [ ]:
softmax(category_2).tolist()


In [ ]:
label_2 = df_cluster_2["encodded_category"]
label_2

In [120]:
log_loss(label_2,softmax(category_2).tolist())

0.8432602584501069

softmax and categorical cross entropy - cluster 3

In [ ]:
le=LabelEncoder()
df_cluster_3["encodded_category"] = le.fit_transform(df_cluster_3["category"])
df_cluster_3

In [ ]:
df_cluster_3= pd.get_dummies(df_cluster_3,columns=["category"],prefix = ["category"])
df_cluster_3

In [ ]:
category_3 = df_cluster_3.iloc[:,5:8].values.tolist()
category_3

In [ ]:
softmax(category_3).tolist()


In [ ]:
label_3 = df_cluster_3["encodded_category"]
label_3

In [127]:
log_loss(label_3,softmax(category_3).tolist())

0.7227430008285542